## NLP
## Lab 4 - Multiword expressions identification and extraction

In [37]:
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
import os
from collections import Counter
import math
import pandas as pd

#### Zadanie 1.
Inicjalizacja tokenizera i wczytanie ustaw

In [2]:
nlp = Polish()
tokenizer = Tokenizer(nlp.vocab)

In [3]:
all_files = []
path = './ustawy'
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r', encoding='utf-8') as file:
        all_files.append(" ".join(file.read().lower().replace('\n', ' ').split()))

#### Zadanie 2.
Przygotowana została uniwersala funkcja pozwalająca zliczyć wystąpienia n-gramów dla dowolnego n.\
Do dalszych zadań przydatna również będzie funkcja wyświetlająca podany zakres wpisów w słowniku oraz zliczone ilości wystąpień pojedynczych słów.

In [4]:
def get_n_grams_for_all_files(n):
    ngrams_frequency_list = {}
    for file in all_files:
        tokens = tokenizer(file)
        token_texts = [token.text for token in tokens]
        ngrams = [" ".join(token_texts[start_pos:start_pos+n]) for start_pos in range(len(tokens) - n + 1)]
        ngrams_counter = Counter(ngrams)
        for ngram, count in ngrams_counter.items():
            if ngram in ngrams_frequency_list:
                ngrams_frequency_list[ngram] += count
            else:
                ngrams_frequency_list[ngram] = count
    return ngrams_frequency_list

In [5]:
def get_entries_range(dictionary, start, end):
    return {k: dictionary[k] for k in list(dictionary)[start:end]}

In [6]:
global_frequency_list = {}
for file in all_files:
    tokens = tokenizer(file)
    words = [token.text for token in tokens if all(char.isalpha() for char in token.text)]
    word_counter = Counter(words)
    for word, count in word_counter.items():
        if word in global_frequency_list:
            global_frequency_list[word] += count
        else:
            global_frequency_list[word] = count

Właściwe obliczenie częstotliwości występowania bigramów odbywa się poniżej.

In [7]:
bigrams = get_n_grams_for_all_files(2)

In [8]:
get_entries_range(bigrams, 0, 15)

{'dz.u. z': 1219,
 'z 1993': 535,
 '1993 r.': 774,
 'r. nr': 17855,
 'nr 129,': 136,
 '129, poz.': 134,
 'poz. 599': 1,
 '599 ustawa': 1,
 'ustawa z': 1187,
 'z dnia': 9513,
 'dnia 9': 216,
 '9 grudnia': 7,
 'grudnia 1993': 110,
 'r. o': 7079,
 'o zmianie': 1297}

#### Zadanie 3.

In [9]:
def is_ngram_alpha(ngram):
    return all(word.isalpha() for word in ngram.split(' '))

In [10]:
letters_bigrams = {bigram : count for bigram, count in bigrams.items() if is_ngram_alpha(bigram)}

In [60]:
get_entries_range(letters_bigrams, 0, 10)

{'ustawa z': 1187,
 'z dnia': 9513,
 'o zmianie': 1297,
 'zmianie ustawy': 850,
 'ustawy o': 1440,
 'o podatku': 533,
 'podatku od': 381,
 'od towarów': 270,
 'towarów i': 530,
 'i usług': 379}

In [61]:
sorted_letters_bigrams = {k: v for k, v in sorted(letters_bigrams.items(), key=lambda item: item[1]*-1)}
get_entries_range(sorted_letters_bigrams, 0, 10)

{'mowa w': 28450,
 'których mowa': 13847,
 'o których': 13839,
 'z dnia': 9513,
 'którym mowa': 9165,
 'o którym': 9155,
 'do spraw': 8680,
 'i nr': 8435,
 'dodaje się': 8190,
 'w drodze': 7092}

In [12]:
get_entries_range(global_frequency_list, 0, 10)

{'z': 81806,
 'nr': 44918,
 'ustawa': 3175,
 'dnia': 17617,
 'grudnia': 2107,
 'o': 64281,
 'zmianie': 1467,
 'ustawy': 9581,
 'podatku': 1806,
 'od': 16603}

#### Zadanie 4.

In [13]:
def pmi(bigram):
    all_words_count = sum(global_frequency_list.values())
    word1, word2 = bigram.split(' ')
    word1_probability = global_frequency_list[word1] / all_words_count
    word2_probability = global_frequency_list[word2] / all_words_count
    bigram_probability = letters_bigrams[bigram] / all_words_count
    return math.log(bigram_probability / float(word1_probability * word2_probability), 10)

In [14]:
bigrams_pmi = {bigram: pmi(bigram) for bigram in letters_bigrams.keys()}

In [33]:
get_entries_range(bigrams_pmi, 0, 10)

{'ustawa z': 1.127910499308115,
 'z dnia': 1.2875890523896676,
 'o zmianie': 1.6064159057351242,
 'zmianie ustawy': 2.2495666395307023,
 'ustawy o': 0.8368576954236884,
 'o podatku': 1.129915506543412,
 'podatku od': 1.5720093296204374,
 'od towarów': 1.4936237092269171,
 'towarów i': 1.0571762531674593,
 'i usług': 1.0121481734524016}

#### Zadanie 5.

In [15]:
sorted_bigrams_pmi = {k: v for k, v in sorted(bigrams_pmi.items(), key=lambda item: item[1]*-1)}

In [16]:
get_entries_range(sorted_bigrams_pmi, 0, 10)

{'kołowe jednoosiowe': 6.46798866782558,
 'automatyki grzewczej': 6.46798866782558,
 'prefabrykatów wnętrzowe': 6.46798866782558,
 'gołe aluminiowe': 6.46798866782558,
 'polistyrenu spienionego': 6.46798866782558,
 'objaśnieniem figur': 6.46798866782558,
 'wkładzie wnoszonym': 6.46798866782558,
 'doktorem habilitowanym': 6.46798866782558,
 'naruszonymi plombami': 6.46798866782558,
 'uw zględnieniu': 6.46798866782558}

#### Zadanie 6.

In [17]:
sorted_bigrams_pmi_five_occ = {bigram : pmi for bigram, pmi in sorted_bigrams_pmi.items() if letters_bigrams[bigram]>=5}

In [18]:
get_entries_range(sorted_bigrams_pmi_five_occ, 0, 10)

{'świeckie przygotowujące': 5.769018663489562,
 'ręcznego miotacza': 5.769018663489562,
 'młyny kulowe': 5.769018663489562,
 'najnowszych zdobyczy': 5.769018663489562,
 'zaszkodzić wynikom': 5.769018663489562,
 'grzegorz schetyna': 5.769018663489562,
 'mleczka makowego': 5.689837417441937,
 'chrześcijan baptystów': 5.689837417441937,
 'przeponowe rurowe': 5.689837417441937,
 'chromu sześciowartościowego': 5.689837417441937}

#### Zadanie 7.

In [19]:
def shannon_entropy(k_list):
    n = sum(k_list)
    return sum([k / n * math.log(abs(k) / n + (k == 0)) for k in k_list])

In [20]:
all_bigrams_occ = sum(letters_bigrams.values())
def llr(bigram):
    word1, word2 = bigram.split(' ')
    k11 = letters_bigrams[bigram]
    k21 = global_frequency_list[word1] - letters_bigrams[bigram]
    k12 = global_frequency_list[word2] - letters_bigrams[bigram]
    k22 = all_bigrams_occ - global_frequency_list[word1] - global_frequency_list[word2]
    return 2 * sum([k11, k12, k21, k22]) * (shannon_entropy([k11, k12, k21, k22]) - shannon_entropy([k11 + k12, k21 + k22]) - shannon_entropy([k11 + k21, k12 + k22]))

In [21]:
bigrams_llr = {bigram: llr(bigram) for bigram in letters_bigrams.keys()}

In [34]:
get_entries_range(bigrams_llr, 0, 10)

{'ustawa z': 3910.0304478605754,
 'z dnia': 40967.65311277764,
 'o zmianie': 8295.061471581146,
 'zmianie ustawy': 7424.721379566656,
 'ustawy o': 2707.13369280242,
 'o podatku': 1710.9035582462352,
 'podatku od': 1935.0128131680278,
 'od towarów': 1264.8425804158696,
 'towarów i': 1564.3556161547951,
 'i usług': 1032.197314412582}

#### Zadanie 8.

In [22]:
sorted_bigrams_llr = {k: v for k, v in sorted(bigrams_llr.items(), key=lambda item: item[1]*-1)}

In [23]:
get_entries_range(sorted_bigrams_llr, 0, 10)

{'mowa w': 139504.46726754995,
 'których mowa': 110489.85516758195,
 'o których': 83375.27141508946,
 'którym mowa': 71378.20475734424,
 'dodaje się': 64460.971993500956,
 'do spraw': 57406.47283185411,
 'o którym': 54695.749116354724,
 'na podstawie': 50161.56449043455,
 'minister właściwy': 46104.13504921431,
 'stosuje się': 44512.12820816197}

#### Zadanie 9.

In [24]:
trigrams = get_n_grams_for_all_files(3)

In [62]:
letters_trigrams = {trigram : count for trigram, count in trigrams.items() if is_ngram_alpha(trigram)}
sorted_letters_trigrams = {k: v for k, v in sorted(letters_trigrams.items(), key=lambda item: item[1]*-1)}
get_entries_range(sorted_letters_trigrams, 0, 10)

{'o których mowa': 13804,
 'których mowa w': 13788,
 'którym mowa w': 9145,
 'o którym mowa': 9138,
 'o której mowa': 5487,
 'której mowa w': 5486,
 'właściwy do spraw': 4609,
 'minister właściwy do': 4277,
 'ustawie z dnia': 3648,
 'w ustawie z': 3644}

#### Zadanie 10.

Analogicznie do zadań, w których obliczane były wartości PMI i LLR dla bigramów, zdefiniowane zostały funkcje dla trigramów. W miejscach gdzie wykorzystywane były pojedyncze słowa bigramu znajdują się pierwszy i drugi bigram zawarty w trigramie, a w miejsce bigramu wstawiony został trigram.

In [27]:
def pmi_trigram(trigram):
    all_bigrams_count = sum(letters_bigrams.values())
    word1, word2, word3 = trigram.split(' ')
    bigram1_probability = letters_bigrams[' '.join([word1, word2])] / all_bigrams_count
    bigram2_probability = letters_bigrams[' '.join([word2, word3])] / all_bigrams_count
    trigram_probability = letters_trigrams[trigram] / all_bigrams_count
    return math.log(trigram_probability / float(bigram1_probability * bigram2_probability), 10)

In [28]:
trigrams_pmi = {trigram: pmi_trigram(trigram) for trigram in letters_trigrams.keys()}
sorted_trigrams_pmi = {k: v for k, v in sorted(trigrams_pmi.items(), key=lambda item: item[1]*-1)}

In [35]:
sorted_trigrams_pmi_five_occ = {trigram: pmi for trigram, pmi in sorted_trigrams_pmi.items() if letters_trigrams[trigram]>=5}

In [36]:
get_entries_range(sorted_trigrams_pmi_five_occ, 0, 15)

{'bankowym postępowaniem ugodowym': 5.668455062795804,
 'przysługuje miesięczny dodatek': 5.668455062795804,
 'o współwłasności w': 5.668455062795804,
 'uroczyście na powierzonym': 5.668455062795804,
 'na powierzonym mi': 5.668455062795804,
 'powierzonym mi stanowisku': 5.668455062795804,
 'dniach tygodnia ustala': 5.668455062795804,
 'dni ustawowo wolnych': 5.668455062795804,
 'zamawiający negocjuje warunki': 5.668455062795804,
 'zostało im cofnięte': 5.668455062795804,
 'im cofnięte pozwolenie': 5.668455062795804,
 'miejsca wyznaczonego lub': 5.668455062795804,
 'wyznaczonego lub uznanego': 5.668455062795804,
 'całości uiszczona przed': 5.668455062795804,
 'zanim zapadł pierwszy': 5.668455062795804}

In [30]:
all_trigrams_occ = sum(letters_trigrams.values())
def llr_trigram(trigram):
    word1, word2, word3 = trigram.split(' ')
    bigram1 = ' '.join([word1, word2])
    bigram2 = ' '.join([word2, word3])
    k11 = letters_trigrams[trigram]
    k21 = letters_bigrams[bigram1] - letters_trigrams[trigram]
    k12 = letters_bigrams[bigram2] - letters_trigrams[trigram]
    k22 = all_trigrams_occ - letters_bigrams[bigram1] - letters_bigrams[bigram2]
    return 2 * sum([k11, k12, k21, k22]) * (shannon_entropy([k11, k12, k21, k22]) - shannon_entropy([k11 + k12, k21 + k22]) - shannon_entropy([k11 + k21, k12 + k22]))

In [31]:
trigrams_llr = {trigram: llr_trigram(trigram) for trigram in letters_trigrams.keys()}
sorted_trigrams_llr = {k: v for k, v in sorted(trigrams_llr.items(), key=lambda item: item[1]*-1)}

In [32]:
get_entries_range(sorted_trigrams_llr, 0, 10)

{'o których mowa': 162222.26329709802,
 'których mowa w': 122962.19274949508,
 'o którym mowa': 115087.86482311715,
 'którym mowa w': 79763.72362136566,
 'o której mowa': 74578.1357189151,
 'minister właściwy do': 57145.696293026944,
 'właściwy do spraw': 50876.30948911843,
 'w ustawie z': 47948.704625061764,
 'której mowa w': 46976.721190722354,
 'ustawie z dnia': 40094.39400940854}

#### Zadanie 11.

In [51]:
def show_as_table(ngrams_results):
    return pd.DataFrame([get_entries_range(ngrams_results, 0, 10).keys(), get_entries_range(ngrams_results, 0, 10).values()]).T

In [56]:
show_as_table(sorted_bigrams_pmi_five_occ)

,0,1
0,świeckie przygotowujące,5.76902
1,ręcznego miotacza,5.76902
2,młyny kulowe,5.76902
3,najnowszych zdobyczy,5.76902
4,zaszkodzić wynikom,5.76902
5,grzegorz schetyna,5.76902
6,mleczka makowego,5.68984
7,chrześcijan baptystów,5.68984
8,przeponowe rurowe,5.68984
9,chromu sześciowartościowego,5.68984


In [57]:
show_as_table(sorted_bigrams_llr)

,0,1
0,mowa w,139504
1,których mowa,110490
2,o których,83375.3
3,którym mowa,71378.2
4,dodaje się,64461
5,do spraw,57406.5
6,o którym,54695.7
7,na podstawie,50161.6
8,minister właściwy,46104.1
9,stosuje się,44512.1


In [58]:
show_as_table(sorted_trigrams_pmi_five_occ)

,0,1
0,bankowym postępowaniem ugodowym,5.66846
1,przysługuje miesięczny dodatek,5.66846
2,o współwłasności w,5.66846
3,uroczyście na powierzonym,5.66846
4,na powierzonym mi,5.66846
5,powierzonym mi stanowisku,5.66846
6,dniach tygodnia ustala,5.66846
7,dni ustawowo wolnych,5.66846
8,zamawiający negocjuje warunki,5.66846
9,zostało im cofnięte,5.66846


In [59]:
show_as_table(sorted_trigrams_llr)

,0,1
0,o których mowa,162222
1,których mowa w,122962
2,o którym mowa,115088
3,którym mowa w,79763.7
4,o której mowa,74578.1
5,minister właściwy do,57145.7
6,właściwy do spraw,50876.3
7,w ustawie z,47948.7
8,której mowa w,46976.7
9,ustawie z dnia,40094.4


Metoda PMI najwyżej punktowała n-gramy, które występowały rzadko, a jednocześnie słowa, z których te n-gramy się składały najczęściej lub zawsze występowały w tekstach jako ten konkretny n-gram.
Metoda LLR z kolei najwyższe wyniki przyznawała n-gramom, które występowały w tekstach najczęściej. Ranking posortowanych n-gramów jest bardzo zbliżony do rankingu posortowanego względem ilości wystąpień.
Używana metoda zależy więc od celu poszukiwań. Jeśli szukamy słów silnie związanych ze sobą, które poza określonymi n-gramami występują bardzo rzadko to lepsze może okazać się PMI, natomiast dla n-gramów charakterystycznych dla ogółu tekstu prawdopodobnie skuteczniej sprawdzi się LLR. 

#### Zadanie 12.

W sekwencjach tokenów nie ma informacji o słowach rozdzielonych znakami interpunkcyjnymi, np. znajdującymi się w dwóch różnych zdaniach, zupełnie niepowiązanymi ze sobą. Częstotliwość występowania bigramów wyznaczona w ten sposób mogłaby zostać zaburzona przez wyrazy często kończące lub rozpoczynające zdania.

Zarówno PMI, jak i LLR działają podobnie dla bigramów i trigramów (opis w zadaniu 11.). W przypadku PMI warto ustawić minimalną częstotliwość wystąpień danego n-gramu, gdyż w przeciwnym przypadku prawdopodobnie największą wartość PMI otrzymają te występujące tylko raz lub bardzo niewiele razy.

PMI znajduje wyrażenia rzadkie, których słowa składowe występują najczęściej właśnie w postaci takiego n-gramu. 
LLR znajduje wyrażenia charakterystyczne dla danego tekstu, pojawiające się w nim często. W tym przypadku są to zwroty cechujące teksty prawne.